<a href="https://colab.research.google.com/github/NishaMurarka11/legal_NER/blob/main/final_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/legal_NER

/content/drive/.shortcut-targets-by-id/1tUg8HPB8MnD8RMVc79KRYvllO_hRSxkf/legal_NER


In [ ]:
# from keras.preprocessing.text import Tokenizer
# from keras_preprocessing.sequence import pad_sequences
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Flatten
# from keras.layers import Embedding, Bidirectional
import pandas as pd
# import numpy as np
# from keras.layers import LSTM
# from keras.layers import TimeDistributed, Activation, Dense
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support

In [ ]:
import json
 
train_judgement_data =  open('dataset/train_data/NER_TRAIN_JUDGEMENT.json', 'r') 
judgement_data = json.load(train_judgement_data)
print(len(judgement_data))

train_preamble_data = open('dataset/train_data/NER_TRAIN_PREAMBLE.json', 'r') 
preamble_data = json.load(train_preamble_data)
print(len(preamble_data))

9435
1560


Data Visualization

In [ ]:
judgement_annotations = pd.DataFrame.from_dict(pd.json_normalize(judgement_data[0]['annotations'][0]['result']), orient='columns')
judgement_annotations

,id,from_name,to_name,type,value.start,value.end,value.text,value.labels
0,C8HPTIM1,label,text,labels,90,103,Hongkong Bank,[ORG]
1,KOWE3RAM,label,text,labels,267,278,Rahul & Co.,[ORG]


In [ ]:
preamble_annotations = pd.DataFrame.from_dict(pd.json_normalize(preamble_data[0]['annotations'][0]['result']), orient='columns')
preamble_annotations

,id,from_name,to_name,type,value.start,value.end,value.text,value.labels
0,2JN4GRYC,label,text,labels,7,40,High Court Of Kerala At Ernakulam,[COURT]
1,SIMZ3AEN,label,text,labels,73,83,T.R.Ajayan,[PETITIONER]
2,S9JF9CCI,label,text,labels,171,182,M.Ravindran,[RESPONDENT]
3,XUXELOF9,label,text,labels,237,251,Nirmala Dinesh,[RESPONDENT]
4,60NAL9QX,label,text,labels,304,311,A.Kumar,[LAWYER]
5,J5CXTLAN,label,text,labels,350,365,M.K.Pushpalatha,[LAWYER]
6,0II679BV,label,text,labels,391,400,P.R.Raman,[JUDGE]
7,G17U4A5Z,label,text,labels,425,436,V.K.Mohanan,[JUDGE]


Splitting the data

In [ ]:
!python -m spacy download en_core_web_lg
!python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 17 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 460.3 MB 27 kB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 1.1 MB 10.1 MB/s 
     |████████████████████████████████| 4.7 MB 63.8 MB/s 
     |████████████████████████████████| 182 kB 75.6 MB/s 
     |████████████████████████████████| 6.6 MB 49.6 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [ ]:
!python -m spacy train training/cfg_albert.cfg --output ./output --paths.train training/Combined_Data_Judgement_Preamble_spacy_format/train.spacy --paths.dev training/Combined_Data_Judgement_Preamble_spacy_format/dev.spacy --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-12-12 04:47:35,025] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-12-12 04:47:35,036] [INFO] Pipeline: ['transformer', 'ner']
INFO:spacy:Pipeline: ['transformer', 'ner']
[2022-12-12 04:47:35,039] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-12-12 04:47:35,041] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.decoder.bias', 'predictions.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.weight', 'predictions.dense.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForS

In [ ]:
import spacy
import urllib
albert_ner = spacy.load("output/model-best")
classes= ['COURT', "PETITIONER", "RESPONDENT", "JUDGE", "LAWYER", "DATE", "ORG", "GPE", 
                              "STATUTE", "PROVISION", "PRECEDENT", "CASE_NUMBER", "WITNESS", "OTHER_PERSON"]
judgment_text = urllib.request.urlopen('https://raw.githubusercontent.com/OpenNyAI/Opennyai/master/samples/sample_judgment1.txt').read().decode()
doc = albert_ner(judgment_text)
colors = {'COURT': "#bbabf2", 'PETITIONER': "#f570ea", "RESPONDENT": "#cdee81", 'JUDGE': "#fdd8a5",
              "LAWYER": "#f9d380", 'WITNESS': "violet", "STATUTE": "#faea99", "PROVISION": "yellow",
              'CASE_NUMBER': "#fbb1cf", "PRECEDENT": "#fad6d6", 'DATE': "#b1ecf7", 'OTHER_PERSON': "#b0f6a2",
              'ORG': '#a57db5', 'GPE': '#7fdbd4'}
options = {"ents": classes, "colors": colors}
spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [ ]:
!python -m spacy evaluate output/model-best training/Combined_Data_Judgement_Preamble_spacy_format/dev.spacy --output output/metrics.json --gold-preproc --gpu-id 0

ℹ Using GPU: 0

================================== Results ==================================

TOK     100.00
NER P   55.06 
NER R   65.98 
NER F   60.03 
SPEED   3752  


=============================== NER (per type) ===============================

                   P       R       F
COURT          85.56   84.95   85.25
DATE           66.04   98.60   79.10
CASE_NUMBER    16.37   73.33   26.76
PETITIONER     54.32   51.76   53.01
OTHER_PERSON   29.09   76.72   42.18
LAWYER         96.03   29.23   44.82
RESPONDENT     79.27   25.00   38.01
JUDGE          81.07   88.96   84.83
STATUTE        80.66   91.59   85.78
WITNESS        58.23   82.14   68.15
ORG            36.90   61.18   46.04
GPE            49.56   63.48   55.67
PROVISION      78.33   91.80   84.53
PRECEDENT      48.10   68.47   56.51

✔ Saved results to output/metrics.json


In [ ]:
!python -m spacy train training/tok2vec.cfg --output ./output_tok --paths.train training/Combined_Data_Judgement_Preamble_spacy_format/train.spacy --paths.dev training/Combined_Data_Judgement_Preamble_spacy_format/dev.spacy --gpu-id 0

ℹ Saving to output directory: output_tok
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-12-12 05:23:23,138] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-12-12 05:23:23,149] [INFO] Pipeline: ['tok2vec', 'ner']
INFO:spacy:Pipeline: ['tok2vec', 'ner']
[2022-12-12 05:23:23,153] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-12-12 05:23:23,154] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
[2022-12-12 05:23:48,648] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
INFO:spacy:Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0

In [ ]:
import spacy
import urllib
tok2_vec_ner = spacy.load("output_tok/model-best")
classes= ['COURT', "PETITIONER", "RESPONDENT", "JUDGE", "LAWYER", "DATE", "ORG", "GPE", 
                              "STATUTE", "PROVISION", "PRECEDENT", "CASE_NUMBER", "WITNESS", "OTHER_PERSON"]
judgment_text = urllib.request.urlopen('https://raw.githubusercontent.com/OpenNyAI/Opennyai/master/samples/sample_judgment1.txt').read().decode()
colors = {'COURT': "#bbabf2", 'PETITIONER': "#f570ea", "RESPONDENT": "#cdee81", 'JUDGE': "#fdd8a5",
              "LAWYER": "#f9d380", 'WITNESS': "violet", "STATUTE": "#faea99", "PROVISION": "yellow",
              'CASE_NUMBER': "#fbb1cf", "PRECEDENT": "#fad6d6", 'DATE': "#b1ecf7", 'OTHER_PERSON': "#b0f6a2",
              'ORG': '#a57db5', 'GPE': '#7fdbd4'}
doc = tok2_vec_ner(judgment_text)
options = {"ents": classes, "colors": colors}
spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [ ]:
!python -m spacy evaluate output_tok/model-best training/Combined_Data_Judgement_Preamble_spacy_format/dev.spacy --output output_tok/metrics.json --gold-preproc --gpu-id 0

ℹ Using GPU: 0

================================== Results ==================================

TOK     100.00
NER P   78.79 
NER R   67.62 
NER F   72.77 
SPEED   46937 


=============================== NER (per type) ===============================

                   P       R       F
COURT          90.51   82.08   86.09
PETITIONER     71.21   55.29   62.25
LAWYER         89.77   79.64   84.40
RESPONDENT     77.78   37.69   50.78
JUDGE          85.03   81.17   83.06
STATUTE        92.11   81.78   86.63
OTHER_PERSON   63.16   59.54   61.30
ORG            52.00   42.76   46.93
DATE           91.58   86.05   88.73
PROVISION      92.53   87.11   89.74
PRECEDENT      55.12   63.06   58.82
GPE            64.58   34.83   45.26
CASE_NUMBER    31.13   44.00   36.46
WITNESS        94.74   64.29   76.60

✔ Saved results to output_tok/metrics.json


In [ ]:
!zip -r output.zip output/

  adding: output/ (stored 0%)
  adding: output/model-last/ (stored 0%)
  adding: output/model-last/tokenizer (deflated 81%)
  adding: output/model-last/meta.json (deflated 66%)
  adding: output/model-last/config.cfg (deflated 61%)
  adding: output/model-last/transformer/ (stored 0%)
  adding: output/model-last/transformer/cfg (stored 0%)
  adding: output/model-last/transformer/model (deflated 11%)
  adding: output/model-last/ner/ (stored 0%)
  adding: output/model-last/ner/model (deflated 8%)
  adding: output/model-last/ner/moves (deflated 75%)
  adding: output/model-last/ner/cfg (deflated 33%)
  adding: output/model-last/vocab/ (stored 0%)
  adding: output/model-last/vocab/strings.json (deflated 76%)
  adding: output/model-last/vocab/vectors (deflated 45%)
  adding: output/model-last/vocab/key2row (stored 0%)
  adding: output/model-last/vocab/vectors.cfg (stored 0%)
  adding: output/model-last/vocab/lookups.bin (stored 0%)
  adding: output/model-best/ (stored 0%)
  adding: output/mode

In [ ]:
!zip -r output_tok.zip output_tok/

  adding: output_tok/ (stored 0%)
  adding: output_tok/metrics.json (deflated 64%)
  adding: output_tok/model-last/ (stored 0%)
  adding: output_tok/model-last/tokenizer (deflated 81%)
  adding: output_tok/model-last/meta.json (deflated 65%)
  adding: output_tok/model-last/config.cfg (deflated 61%)
  adding: output_tok/model-last/tok2vec/ (stored 0%)
  adding: output_tok/model-last/tok2vec/cfg (stored 0%)
  adding: output_tok/model-last/tok2vec/model (deflated 7%)
  adding: output_tok/model-last/ner/ (stored 0%)
  adding: output_tok/model-last/ner/model (deflated 7%)
  adding: output_tok/model-last/ner/moves (deflated 75%)
  adding: output_tok/model-last/ner/cfg (deflated 33%)
  adding: output_tok/model-last/vocab/ (stored 0%)
  adding: output_tok/model-last/vocab/strings.json (deflated 76%)
  adding: output_tok/model-last/vocab/vectors (deflated 45%)
  adding: output_tok/model-last/vocab/key2row (stored 0%)
  adding: output_tok/model-last/vocab/vectors.cfg (stored 0%)
  adding: output